In [9]:
import pandas as pd
from dfply import *

import sys
from sshtunnel import SSHTunnelForwarder
import logging
sys.path.append('/Users/sdc/Documents/bida_lab/db/P1_extubationfailure/서대철')

import src.data_extraction.access_database as db   # DB 연결 관련 모듈

## Features: Severity (Charlson, APS III, SAPS II, OASIS)

### 1. Subjectlist 가져오기

In [8]:
# 데이터 저장위치
# data_dir = './data'   # 수정 쌤 디렉토리에서 사용
data_dir = '../../outputs'   # 내 디렉에서 사용
sub_list_dir = data_dir + '/' + 'sub1list.csv'


subjectlist1 = pd.read_csv(sub_list_dir, parse_dates=['admittime', 'dischtime'])
sub1list = subjectlist1 >> select('subject_id','hadm_id','stay_id', 'admittime', 
                                'dischtime', 'gender', 'age', 'height', 
                                'weight')
sub1list = sub1list.drop_duplicates(subset='stay_id')   # remove dup
sub1list

,subject_id,hadm_id,stay_id,admittime,dischtime,gender,age,height,weight
0,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-20 05:15:00,F,77.018296,157.0,65.0
1,10003400,23559586,38383343,2137-08-04 00:07:00,2137-09-02 17:05:00,F,75.589414,165.0,99.6
2,10004401,27939719,31202136,2144-04-11 03:31:00,2144-04-13 17:31:00,M,85.274942,NaN,80.4
3,10004401,29988601,32773003,2144-01-23 07:58:00,2144-02-06 11:45:00,M,85.059155,168.0,76.0
4,10005817,28661809,31316840,2135-01-03 21:54:00,2135-01-19 18:36:00,M,69.008724,175.0,88.8
...,...,...,...,...,...,...,...,...,...
7226,19995595,21784060,34670930,2126-10-17 21:43:00,2126-11-12 17:35:00,M,68.793734,163.0,94.7
7227,19997367,20617667,35616526,2126-04-20 07:15:00,2126-05-19 14:15:00,F,63.299259,147.0,59.0
7228,19999068,21606769,30143796,2161-08-24 04:10:00,2161-09-02 19:00:00,M,63.643884,170.0,55.8
7229,19999442,26785317,32336619,2148-11-19 10:00:00,2148-12-04 16:25:00,M,43.884161,193.0,107.5


### 2. DB 연결 후 필요한 테이블 가져오기
- charlson
- apsiii
- sapsii
- oasis


In [10]:
# 1. DB 파라미터 설정

# 데이터 저장위치
# output_dir = './data'   # 수정 쌤 디렉토리에서 사용
output_dir = '../../outputs'   # 내 디렉에서 사용


# SSH 연결 설정
ssh_config = {
    'ssh_ip': '210.91.223.248',
    'ssh_port': 35430,
    'ssh_username': "medicalai",
    'ssh_password': "medicalai731!",
    'remote_bind_address': ('localhost', 35432)  # 데이터베이스 호스트 및 포트
}

# 데이터베이스 연결 설정
db_config = {
    'database': 'mimiciv',
    'user': 'mai_admin',
    'password': 'admin1q2w!@',
    'host': 'localhost',  # SSH 터널을 사용하면, 이 값은 'localhost'가 됩니다
    'port': 35432  # 이 값은 SSH 터널 로컬 포트 바인딩에 의해 대체됩니다
}


# 테이블 쿼리 설정
tables_query = {
    'charlson': "select * from mimiciv_derived.charlson;",
    'apsiii': "select * from mimiciv_derived.apsiii;",
    'sapsii': "select * from mimiciv_derived.sapsii;",
    'oasis': "select * from mimiciv_derived.oasis;"
                }


db.print_config_info(db_config, tables_query)

--------- Database Configuration ---------
Database Name: mimiciv
User: mai_admin
Password: ***********
Host: localhost
Port: 35432

--------- SQL Queries for Required Tables ---------
charlson: select * from mimiciv_derived.charlson;
apsiii: select * from mimiciv_derived.apsiii;
sapsii: select * from mimiciv_derived.sapsii;
oasis: select * from mimiciv_derived.oasis;



In [11]:
# 2. db 접속, 테이블 추출, 'dataframes' 라는 딕셔너리에 테이블 저장

try:
    conn, tunnel = db.connect_to_database_via_ssh(db_config, ssh_config)
    if conn is not None:
        # 데이터베이스 작업 수행
        dataframes = db.retrieve_data(conn, tables_query)
    else:
        logging.error("Failed to connect to the database.")
except Exception as e:
    logging.error(f"An error occurred: {e}")
finally:
    if conn is not None:
        db.disconnect_database(conn)


# 데이터프레임 변수로 저장
## Severity
charlson = dataframes['charlson']
apsiii = dataframes['apsiii']
sapsii = dataframes['sapsii']
oasis = dataframes['oasis']


INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.2p1)
INFO:paramiko.transport:Authentication (password) successful!
INFO:root:SSH TUNNEL ESTABLISHED...
INFO:root:mimiciv DATABASE CONNECTED VIA SSH.
INFO:root:Retrieved charlson: (431231, 21)
INFO:root:Retrieved apsiii: (73181, 21)
INFO:root:Retrieved sapsii: (73181, 22)
INFO:root:Retrieved oasis: (73181, 25)
INFO:root:DATABASE CONNCETION CLOSED.


In [13]:
charlson

,subject_id,hadm_id,age_score,myocardial_infarct,congestive_heart_failure,peripheral_vascular_disease,cerebrovascular_disease,dementia,chronic_pulmonary_disease,rheumatic_disease,...,mild_liver_disease,diabetes_without_cc,diabetes_with_cc,paraplegia,renal_disease,malignant_cancer,severe_liver_disease,metastatic_solid_tumor,aids,charlson_comorbidity_index
0,18910522,20000041,2,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,3
1,14546051,20000069,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,19618591,20000200,2,1,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,5
3,16179342,20000298,3,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,5
4,15325429,20000400,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431226,14351952,29999277,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
431227,17128602,29999326,2,0,1,0,0,0,1,0,...,1,1,0,0,0,1,0,0,0,8
431228,10053207,29999444,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
431229,10698563,29999501,3,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5


### 3. 테이블 결합하기

In [14]:
# charlson comorbidity index (CCI) 결합 (hadm_id 기준)
sub1list_cci = pd.merge(sub1list, charlson[['subject_id', 'hadm_id', 'charlson_comorbidity_index']], on=['subject_id', 'hadm_id'], how='left')
sub1list_cci

,subject_id,hadm_id,stay_id,admittime,dischtime,gender,age,height,weight,charlson_comorbidity_index
0,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-20 05:15:00,F,77.018296,157.0,65.0,5
1,10003400,23559586,38383343,2137-08-04 00:07:00,2137-09-02 17:05:00,F,75.589414,165.0,99.6,12
2,10004401,27939719,31202136,2144-04-11 03:31:00,2144-04-13 17:31:00,M,85.274942,NaN,80.4,5
3,10004401,29988601,32773003,2144-01-23 07:58:00,2144-02-06 11:45:00,M,85.059155,168.0,76.0,5
4,10005817,28661809,31316840,2135-01-03 21:54:00,2135-01-19 18:36:00,M,69.008724,175.0,88.8,7
...,...,...,...,...,...,...,...,...,...,...
7226,19995595,21784060,34670930,2126-10-17 21:43:00,2126-11-12 17:35:00,M,68.793734,163.0,94.7,3
7227,19997367,20617667,35616526,2126-04-20 07:15:00,2126-05-19 14:15:00,F,63.299259,147.0,59.0,7
7228,19999068,21606769,30143796,2161-08-24 04:10:00,2161-09-02 19:00:00,M,63.643884,170.0,55.8,3
7229,19999442,26785317,32336619,2148-11-19 10:00:00,2148-12-04 16:25:00,M,43.884161,193.0,107.5,4


In [15]:
apsiii

,subject_id,hadm_id,stay_id,apsiii,apsiii_prob,hr_score,mbp_score,temp_score,resp_rate_score,pao2_aado2_score,...,wbc_score,creatinine_score,uo_score,bun_score,sodium_score,albumin_score,bilirubin_score,glucose_score,acidbase_score,gcs_score
0,12466550,23998182,30000153,44,0.08654575921862385844,7.0,6.0,0.0,0.0,NaN,...,0.0,0.0,5.0,7.0,0.0,NaN,NaN,0.0,1.0,15.0
1,13180007,27543152,30000213,44,0.08654575921862385844,0.0,7.0,0.0,6.0,NaN,...,0.0,7.0,0.0,11.0,2.0,0.0,0.0,5.0,NaN,3.0
2,18421337,22413411,30000484,79,0.33126704000552879910,1.0,15.0,2.0,8.0,NaN,...,5.0,0.0,5.0,11.0,0.0,0.0,0.0,0.0,NaN,29.0
3,12207593,22795209,30000646,29,0.04455469432701904035,1.0,15.0,0.0,9.0,NaN,...,0.0,0.0,1.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0
4,12980335,23552849,30001148,82,0.36339013544336734819,0.0,7.0,2.0,17.0,0.0,...,0.0,0.0,4.0,0.0,0.0,NaN,NaN,0.0,1.0,48.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73176,16180713,20178444,39999301,46,0.09431596692709329452,0.0,10.0,2.0,8.0,0.0,...,0.0,4.0,7.0,7.0,0.0,0.0,0.0,5.0,0.0,0.0
73177,15498623,27161460,39999384,50,0.11174890826305648295,0.0,15.0,0.0,8.0,NaN,...,0.0,7.0,5.0,12.0,0.0,NaN,NaN,0.0,NaN,0.0
73178,11256534,26910235,39999552,81,0.35252925485606272195,7.0,15.0,0.0,6.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,2.0,48.0
73179,15403458,25335698,39999562,14,0.02243681045149745325,5.0,4.0,0.0,NaN,NaN,...,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [16]:
# apsiii 결합 (stay_id 기준). 참고: apsiii, apsiii_prob (사망 확률) 값 존재. apsiii 점수만 결합.
sub1list_apsiii = pd.merge(sub1list_cci, apsiii[['subject_id', 'hadm_id', 'stay_id', 'apsiii']], on=['subject_id', 'hadm_id', 'stay_id'], how='left')
sub1list_apsiii

,subject_id,hadm_id,stay_id,admittime,dischtime,gender,age,height,weight,charlson_comorbidity_index,apsiii
0,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-20 05:15:00,F,77.018296,157.0,65.0,5,51
1,10003400,23559586,38383343,2137-08-04 00:07:00,2137-09-02 17:05:00,F,75.589414,165.0,99.6,12,103
2,10004401,27939719,31202136,2144-04-11 03:31:00,2144-04-13 17:31:00,M,85.274942,NaN,80.4,5,46
3,10004401,29988601,32773003,2144-01-23 07:58:00,2144-02-06 11:45:00,M,85.059155,168.0,76.0,5,101
4,10005817,28661809,31316840,2135-01-03 21:54:00,2135-01-19 18:36:00,M,69.008724,175.0,88.8,7,51
...,...,...,...,...,...,...,...,...,...,...,...
7226,19995595,21784060,34670930,2126-10-17 21:43:00,2126-11-12 17:35:00,M,68.793734,163.0,94.7,3,61
7227,19997367,20617667,35616526,2126-04-20 07:15:00,2126-05-19 14:15:00,F,63.299259,147.0,59.0,7,32
7228,19999068,21606769,30143796,2161-08-24 04:10:00,2161-09-02 19:00:00,M,63.643884,170.0,55.8,3,50
7229,19999442,26785317,32336619,2148-11-19 10:00:00,2148-12-04 16:25:00,M,43.884161,193.0,107.5,4,28


In [17]:
sapsii

,subject_id,hadm_id,stay_id,starttime,endtime,sapsii,sapsii_prob,age_score,hr_score,sysbp_score,...,uo_score,bun_score,wbc_score,potassium_score,sodium_score,bicarbonate_score,bilirubin_score,gcs_score,comorbidity_score,admissiontype_score
0,14046553,20000094,35605481,2150-03-02 15:19:31,2150-03-03 15:19:31,98,0.982729,18,11.0,13.0,...,11.0,10.0,0.0,0.0,0.0,3.0,0.0,26.0,0,6
1,14990224,20000147,30503572,2121-08-30 16:33:54,2121-08-31 16:33:54,35,0.166523,15,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0,6
2,17913090,20000351,30593599,2145-06-13 20:10:27,2145-06-14 20:10:27,24,0.058153,7,0.0,0.0,...,11.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,6
3,16788749,20000808,30610654,2180-01-17 19:32:10,2180-01-18 19:32:10,17,0.025876,7,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0,8
4,16788749,20000808,35191063,2180-01-12 22:07:00,2180-01-13 22:07:00,26,0.071716,7,0.0,0.0,...,11.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73176,14641622,29999186,32199257,2135-07-12 01:51:00,2135-07-13 01:51:00,30,0.106398,7,0.0,5.0,...,0.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0,0,8
73177,10053207,29999444,31112026,2199-12-14 19:52:26,2199-12-15 19:52:26,26,0.071716,7,4.0,0.0,...,0.0,6.0,0.0,0.0,0.0,3.0,0.0,0.0,0,6
73178,13478841,29999498,33874605,2188-05-13 12:03:00,2188-05-14 12:03:00,34,0.152870,16,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,9,6
73179,10355856,29999625,36975675,2157-11-07 11:51:00,2157-11-08 11:51:00,32,0.128048,12,0.0,5.0,...,0.0,0.0,0.0,0.0,1.0,0.0,NaN,0.0,0,8


In [18]:
# sapsii 결합 (stay_id 기준). 참고: sapsii, sapsii_prob (사망 확률) 값 존재. sapsii 점수만 결합.
sub1list_sapsii = pd.merge(sub1list_apsiii, sapsii[['subject_id', 'hadm_id', 'stay_id', 'sapsii']], on=['subject_id', 'hadm_id', 'stay_id'], how='left')
sub1list_sapsii

,subject_id,hadm_id,stay_id,admittime,dischtime,gender,age,height,weight,charlson_comorbidity_index,apsiii,sapsii
0,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-20 05:15:00,F,77.018296,157.0,65.0,5,51,44
1,10003400,23559586,38383343,2137-08-04 00:07:00,2137-09-02 17:05:00,F,75.589414,165.0,99.6,12,103,83
2,10004401,27939719,31202136,2144-04-11 03:31:00,2144-04-13 17:31:00,M,85.274942,NaN,80.4,5,46,43
3,10004401,29988601,32773003,2144-01-23 07:58:00,2144-02-06 11:45:00,M,85.059155,168.0,76.0,5,101,56
4,10005817,28661809,31316840,2135-01-03 21:54:00,2135-01-19 18:36:00,M,69.008724,175.0,88.8,7,51,32
...,...,...,...,...,...,...,...,...,...,...,...,...
7226,19995595,21784060,34670930,2126-10-17 21:43:00,2126-11-12 17:35:00,M,68.793734,163.0,94.7,3,61,36
7227,19997367,20617667,35616526,2126-04-20 07:15:00,2126-05-19 14:15:00,F,63.299259,147.0,59.0,7,32,55
7228,19999068,21606769,30143796,2161-08-24 04:10:00,2161-09-02 19:00:00,M,63.643884,170.0,55.8,3,50,44
7229,19999442,26785317,32336619,2148-11-19 10:00:00,2148-12-04 16:25:00,M,43.884161,193.0,107.5,4,28,14


In [19]:
oasis

,subject_id,hadm_id,stay_id,oasis,oasis_prob,age,age_score,preiculos,preiculos_score,gcs,...,resprate,resp_rate_score,temp,temp_score,urineoutput,urineoutput_score,mechvent,mechvent_score,electivesurgery,electivesurgery_score
0,15726063,23069026,36415879,27,0.06110557118878299662,53.3839872681065644,6,73.0000000000000000,3,15.0,...,31.0,6.0,38.56,2.0,4580.0,0.0,0,0,0,6
1,14304873,20156152,35621403,28,0.06884295828308623481,75.1505757369999434,6,9241.9666666666666667,1,14.0,...,32.0,6.0,35.40,4.0,2390.0,1.0,0,0,0,6
2,12117555,25128945,37212487,34,0.13709860673818602936,82.37789823064038515712,9,14055.783333333333,1,9.0,...,27.0,1.0,36.6150000000000000,0.0,85.0,10.0,0,0,0,6
3,16645325,26608451,37532286,33,0.12270071663757324864,77.62581604951116124530,9,1691.9833333333333333,1,15.0,...,24.0,1.0,38.00,2.0,1015.0,5.0,0,0,0,6
4,10989579,26999802,38267080,26,0.05418719720382744723,25.87166522470033566786,3,95.0000000000000000,3,15.0,...,34.0,6.0,36.39,2.0,1225.0,5.0,0,0,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73176,13282748,21364602,34149913,25,0.04801206627528390922,67.6392433722785928,6,58.0000000000000000,3,9.0,...,27.0,1.0,37.56,2.0,3170.0,0.0,0,0,0,6
73177,11079785,21641322,38019496,24,0.04250902048594646910,46.8008161559854684,3,10803.116666666667,1,15.0,...,43.0,6.0,39.33,2.0,3870.0,0.0,0,0,0,6
73178,10249699,24955103,34378808,24,0.04250902048594646910,52.19953475290963860186,3,23428.933333333333,2,15.0,...,41.0,6.0,35.61,4.0,NaN,NaN,0,0,0,6
73179,17747104,20403718,32048592,21,0.02939485724741537912,29.7838344926864319,3,44.0000000000000000,3,14.0,...,10.0,1.0,37.33,2.0,2400.0,1.0,0,0,0,6


In [21]:
# oasis 결합 (stay_id 기준). 참고: oasis, oasis_prob (사망 확률) 값 존재. oasis 점수만 결합.
sub1list_oasis = pd.merge(sub1list_sapsii, oasis[['subject_id', 'hadm_id', 'stay_id', 'oasis']], on=['subject_id', 'hadm_id', 'stay_id'], how='left')
sub1list_oasis

,subject_id,hadm_id,stay_id,admittime,dischtime,gender,age,height,weight,charlson_comorbidity_index,apsiii,sapsii,oasis
0,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-20 05:15:00,F,77.018296,157.0,65.0,5,51,44,34
1,10003400,23559586,38383343,2137-08-04 00:07:00,2137-09-02 17:05:00,F,75.589414,165.0,99.6,12,103,83,60
2,10004401,27939719,31202136,2144-04-11 03:31:00,2144-04-13 17:31:00,M,85.274942,NaN,80.4,5,46,43,53
3,10004401,29988601,32773003,2144-01-23 07:58:00,2144-02-06 11:45:00,M,85.059155,168.0,76.0,5,101,56,46
4,10005817,28661809,31316840,2135-01-03 21:54:00,2135-01-19 18:36:00,M,69.008724,175.0,88.8,7,51,32,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7226,19995595,21784060,34670930,2126-10-17 21:43:00,2126-11-12 17:35:00,M,68.793734,163.0,94.7,3,61,36,35
7227,19997367,20617667,35616526,2126-04-20 07:15:00,2126-05-19 14:15:00,F,63.299259,147.0,59.0,7,32,55,43
7228,19999068,21606769,30143796,2161-08-24 04:10:00,2161-09-02 19:00:00,M,63.643884,170.0,55.8,3,50,44,46
7229,19999442,26785317,32336619,2148-11-19 10:00:00,2148-12-04 16:25:00,M,43.884161,193.0,107.5,4,28,14,25


In [23]:
sub1list_oasis.info()   # null 값 없는 거 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7231 entries, 0 to 7230
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   subject_id                  7231 non-null   int64         
 1   hadm_id                     7231 non-null   int64         
 2   stay_id                     7231 non-null   int64         
 3   admittime                   7231 non-null   datetime64[ns]
 4   dischtime                   7231 non-null   datetime64[ns]
 5   gender                      7231 non-null   object        
 6   age                         7231 non-null   float64       
 7   height                      5026 non-null   float64       
 8   weight                      6475 non-null   float64       
 9   charlson_comorbidity_index  7231 non-null   int64         
 10  apsiii                      7231 non-null   int64         
 11  sapsii                      7231 non-null   int64       

### 4. 테이블 저장

In [24]:
file_name = output_dir + '/' + 'sub1list_cci_apsii_sapsii_oasis.csv'
sub1list_oasis.to_csv(file_name, index=False, encoding='utf-8-sig')